In [20]:
# Uncomment these lines when running for the first time to install the required packages.  This is mostly required for local development.
# # %pip install openai
# %pip install google-auth
# %pip install google-cloud-bigquery
# %pip install google-cloud-storage
# %pip install lca
# %pip install banjo
# %pip install tqdm
# %pip install oauth2client
# %pip install tqdm.contrib
# %pip install google-generativeai
# %pip install google-genai
# %pip install av

In [21]:
import os
import time
import tempfile
import logging
from datetime import datetime, timedelta, timezone
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

import requests
import pandas as pd
import av
from tqdm import tqdm
from tqdm.contrib.concurrent import thread_map
from functools import partial

from google.cloud import bigquery, storage
from google import genai
from google.api_core.exceptions import TooManyRequests

from banjo import utils
from banjo.utils.shibainu import (
    Classification,
    estimate_run_cost,
    configure_logger,
)

# Import maps viral places classification utils
from utils.constant import get_viral_places_query, place_agg_dict, PLACE_REQUIRED_KEYS, PLACE_LIST_COLS, VIDEO_REQUIRED_KEYS, SELECTED_COLS, STORY_COLS_RENAME
from utils.helper import download_and_upload, topk_by_score_per_place, parse_incident_json_broken, parse_incident_safe, majority_vote, combine_text_list
from utils.prompt import VIDEO_CLASSIFIER_PROMPT, TEXT_CLASSIFIER_PROMPT

# Configure logging
configure_logger(level=logging.ERROR)

<Logger shibainu (ERROR)>

#### Parameters

In [22]:
START_DATE = '20250703'
END_DATE = '20250705'
VIEW_WEIGHT = 0.6
FRESHNESS_WEIGHT = 0.4
SAMPLE_PER_PLACE = 5
MAX_WORKERS = 10

# Initialize GCS client
client = storage.Client(project="myaigcp")
BUCKET_NAME = "shiba-inu-temp"
BUCKET_FOLDER = "maps_events_20250720"
BUCKET = client.bucket(BUCKET_NAME)

# Save Destination 
WRITE_PROJECT_ID = "sc-bq-gcs-billingonly"
WRITE_DATASET = "temp_datascience"
WRITE_TABLE_NAME = "maps_viral_places_classification"

DESTINATION = f"{WRITE_PROJECT_ID}.{WRITE_DATASET}.{WRITE_TABLE_NAME}"
# SERVICE_ACCOUNT = 'shiba-inu@sc-product-datascience.iam.gserviceaccount.com'

#### Import Data and Upload Media URL to GCS 

In [23]:
#Step 1: Import Data
query = get_viral_places_query(START_DATE, END_DATE, VIEW_WEIGHT, FRESHNESS_WEIGHT)
df = utils.gbq.read_gbq(query, 
                    project_id="myaigcp",
                    dialect="standard",
                    priority="interactive")


E0000 00:00:1762276756.669660 35531534 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [ ]:
# Step 2: Download and Upload Media URL to GCS
worker = partial(
    download_and_upload,
    bucket=BUCKET,
    bucket_name=BUCKET_NAME,
    bucket_folder=BUCKET_FOLDER,
    url_col='media_url',
    id_col='story_snap_id',
)
rows = df.to_dict(orient="records")       
with ThreadPoolExecutor(max_workers=10) as ex:
    results = list(tqdm(ex.map(worker, rows), total=len(rows)))

# Write the results back to the original df
df.loc[df.index, "gcs_url"] = results


#### Call Shibainu Classification

In [25]:
# Step 1: Initialize Shibainu Classification
video_classifier = Classification(
    provider_name="gemini",
    model_name='gemini-2.0-flash',
    input_type="video",
    provider_config={"project_id": "myaigcp",
                    "location": "us-central1"}, #this config is used for gemini only
    processor_config = {
        "processing_mode": "image_url",  # "image_url" or "bytes"
        'return_direct_url': True       
    },
    model_parameters={
        "temperature": 0,
        "max_token": 1024
    },
    prompt=VIDEO_CLASSIFIER_PROMPT
)

video_classifier.get_result(video_classifier.send_message('What is your model?'))

'I am a large language model, trained by Google.'

In [26]:
# Step2: Choose top K videos per place and run classification
df_selected_raw = topk_by_score_per_place(df, group_col='place_id', order_col='score', filter_col='gcs_url', k=SAMPLE_PER_PLACE)
video_classifier_results = thread_map(video_classifier.classify, df_selected_raw['gcs_url'].tolist(), max_workers=MAX_WORKERS)

100%|██████████| 49/49 [00:32<00:00,  1.52it/s]


In [27]:
# Step 3: Parse labels and construct the summary table
video_labels = [video_classifier.get_result(r) for r in video_classifier_results]
video_usage  = [video_classifier.get_token_usage(r) for r in video_classifier_results]

df_video_summary = df_selected_raw.copy()
df_video_summary.loc[:, "video_labels"] = video_labels
df_video_summary.loc[:, "video_prompt_tokens"]= [u.get("prompt_tokens") for u in video_usage]
df_video_summary.loc[:, "video_completion_tokens"]= [u.get("completion_tokens") for u in video_usage]

# 3.1 Parse labels
video_parsed = (
    df_video_summary["video_labels"]
    .map(lambda x: parse_incident_safe(x, VIDEO_REQUIRED_KEYS))
    .apply(pd.Series)
)

# 3.2 Attach parsed columns
df_video_summary = pd.concat([df_video_summary, video_parsed], axis=1)

#### Using Text Classification to Consolidate

In [28]:
# Group and aggregate
df_place_raw = df_video_summary.groupby('place_id', as_index=False).agg(place_agg_dict)

In [29]:
# Text classification
text_classifier = Classification(
    provider_name="gemini",
    model_name='gemini-2.0-flash',
    input_type="text",
    provider_config={"project_id": "myaigcp",
                    "location": "us-central1"}, #this config is used for gemini only
    # processor_config = {
    #     "processing_mode": "image_url",  # "image_url" or "bytes"
    #     'return_direct_url': True       
    # },
    model_parameters={
        "temperature": 0,
        "max_token": 2000,
    },
    prompt=TEXT_CLASSIFIER_PROMPT
)

text_results = thread_map(text_classifier.classify, df_place_raw.to_dict("records"), max_workers=10)

100%|██████████| 11/11 [00:04<00:00,  2.53it/s]


In [30]:
# Construct the results
text_labels = [text_classifier.get_result(r) for r in text_results]

df_place_summary = df_place_raw.copy()
df_place_summary.drop(columns=["key_objects_entities", "activity_type", "contributing_context",
                               "short_description", "long_description", "keywords"], inplace=True)
df_place_summary.loc[:, "text_labels"] = text_labels


# Parse labels
text_parsed = (
    df_place_summary["text_labels"]
    .map(lambda x: parse_incident_safe(x, PLACE_REQUIRED_KEYS))
    .apply(pd.Series)
)

df_place_summary = pd.concat([df_place_summary, text_parsed], axis=1)

In [37]:
PLACE_REQUIRED_KEYS

['key_objects_entities',
 'activity_type',
 'short_description',
 'long_description',
 'keywords',
 'consistency']

#### Consolidate the Output

In [32]:
df_video_summary = df_video_summary.rename(
    columns={col: f"video_{col}" for col in STORY_COLS_RENAME}
)
df_merged = df_place_summary.merge(df_video_summary, on=['place_id', 'continent','place_name', 'place_country_code',
                                                        'detection_start_time', 'detection_end_time'], how='left')

#### Write Output to BQ

In [33]:
df_merged.to_gbq(destination_table= DESTINATION, project_id= WRITE_PROJECT_ID, if_exists='replace')
print(f"`{DESTINATION}` was created successfully!")

100%|██████████| 1/1 [00:00<00:00, 6574.14it/s]

`sc-bq-gcs-billingonly.temp_datascience.maps_viral_places_classification` was created successfully!
